# 1. Download audio dataset, set up models

In [3]:
from datasets import load_dataset, Audio
dataset = load_dataset("synthbot/pony-speech")
train_data = dataset['train']

from svc_helper.sfeatures.models import RVCHubertModel, SVC5WhisperModel
import torch
sfeatures_model = RVCHubertModel(device = torch.device('cuda'))
sfeatures_model2 = SVC5WhisperModel(device = torch.device('cuda'))


import librosa
def add_speech_features(example):
    audio = example['audio']['array']
    audio_resamp = librosa.resample(audio,
        orig_sr=example['audio']['sampling_rate'],
        target_sr=RVCHubertModel.expected_sample_rate)
    audio_padded = sfeatures_model.pad_audio(audio_resamp)
    feats = sfeatures_model.extract_features(audio=
        torch.from_numpy(audio_padded))
    feats2 = sfeatures_model2.extract_features(
        audio=torch.from_numpy(audio_padded))
    example['rvc_features'] = feats.cpu().numpy()
    example['whisper_features'] = feats2.cpu().numpy()
    return example

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

2024-08-01 13:53:45 | INFO | fairseq.tasks.hubert_pretraining | current directory is d:\Code\raraai\4_justusewhisper
2024-08-01 13:53:45 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2024-08-01 13:53:45 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_features': 0.1

# 2. Select speakers and extract features (unconditional dataset)

In [4]:
speakers = {'Rarity'}
speakers_data = {}
n_data = 2000

filtered_dataset = train_data.filter(lambda ex, speakers=speakers:
    ex['speaker'] in speakers, num_proc=16).shuffle().select(range(n_data)).map(
        add_speech_features)
filtered_dataset.to_parquet(f'dataset_unconditional_{n_data}.parquet')

c:\Users\vul\AppData\Local\Programs\Python\Python310\lib\site-packages\dill\_dill.py:1705: PicklingWarning: Cannot locate reference to <enum 'Choices'>.
  warnings.warn('Cannot locate reference to %r.' % (obj,), PicklingWarning)
c:\Users\vul\AppData\Local\Programs\Python\Python310\lib\site-packages\dill\_dill.py:1707: PicklingWarning: Cannot pickle <enum 'Choices'>: fairseq.dataclass.constants.Choices has recursive self-references that trigger a RecursionError.
  warnings.warn('Cannot pickle %r: %s.%s has recursive self-references that trigger a RecursionError.' % (obj, obj.__module__, obj_name), PicklingWarning)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

4436685282